In [4]:
import math as m
import pandas as pn
import numpy as np
import random


### ACO

In [17]:
class Graph(object):
    def __init__(self, cost_matrix: list, rank: int):
        """
        :param cost_matrix:
        :param rank: rank of the cost matrix
        
        """
        self.matrix = cost_matrix
        self.rank = rank
        self.pheromone = [[1/rank for j in range(rank)] for i in range(rank)]


class ACO(object):
    def __init__(self, ant_count: int, generations: int, alpha: float, beta: float, rho: float, q: int,
                 strategy: int):
        """
        :param ant_count:
        :param generations:
        :param alpha: relative importance of pheromone
        :param beta: relative importance of heuristic information
        :param rho: pheromone residual coefficient
        :param q: pheromone intensity
        :param strategy: pheromone update strategy. 0 - ant-cycle, 1 - ant-quality, 2 - ant-density
        
        """
        self.Q = q
        self.rho = rho
        self.beta = beta
        self.alpha = alpha
        self.ant_count = ant_count
        self.generations = generations
        self.update_strategy = strategy

    def _update_pheromone(self, graph: Graph, ants: list):
        
        for i, row in enumerate(graph.pheromone):
            for j, col in enumerate(row):
                
                graph.pheromone[i][j] *= self.rho
                
                for ant in ants:
                    graph.pheromone[i][j] += ant.pheromone_delta[i][j]

    
    def solve(self, graph: Graph):
        """
        :param graph:
        
        """
        best_cost = float('-inf')
        best_solution = []
        
        for gen in range(self.generations):
            
            ants = []
            ant = _Ant(self, graph, random.randint(0, graph.rank - 1))
            ants.append(ant)
            
            best = [ant.graph.matrix[ant.start][0], ant.start]
            
            for loop in range(self.ant_count):
                for i in range(graph.rank - 1):
                    cst = ant._select_next()
                    if cst > best[0]:
                        best = [cst, ant.current]
                

                ant.total_cost += graph.matrix[ant.tabu[-1]][ant.tabu[0]]
                if ant.total_cost > best_cost:
                    best_cost = ant.total_cost
                    best_solution = [] + ant.tabu
                    
                    
                # update pheromone
                ant._update_pheromone_delta()
                ant._remember_best_path(best_solution)
                
                if loop != (self.ant_count - 1):
                    ant = _Ant(self, graph,  random.randint(0, graph.rank - 1))
                    ants.append(ant)
            
            
            self._update_pheromone(graph, ants)
            #print('generation #{}, best cost: {}, path: {}'.format(gen, best_cost, best_solution))
        return best_solution, best_cost


class _Ant(object):
    
    def __init__(self, aco: ACO, graph: Graph, start):
        self.colony = aco
        self.graph = graph
        self.total_cost = 0.0
        self.tabu = []  # tabu list
        self.pheromone_delta = []  # the local increase of pheromone
        self.allowed = [i for i in range(graph.rank)]  # nodes which are allowed for the next selection
        self.eta = [[0 if i == j else 1 / graph.matrix[i][j] for j in range(graph.rank)] for i in
                    range(graph.rank)]  # heuristic information
        self.start = start  # start from any node
        self.tabu.append(start)
        self.current = start
        self.allowed.remove(start)

        
        
    def _select_next(self):
        denominator = 0
        
        for i in self.allowed:
            # self.graph.matrix[self.current][i] is added to give more weight in the probability to the nodes with more overlap
            denominator += self.graph.pheromone[self.current][i] ** self.colony.alpha * self.eta[self.current][i] \
                                                    ** self.colony.beta * self.graph.matrix[self.current][i] ** 2
            
            
        probabilities = [0 for i in range(self.graph.rank)]  # probabilities for moving to a node in the next step
        
        for i in range(self.graph.rank):
            
            try:
                self.allowed.index(i)  # test if allowed list contains i
                
                probabilities[i] = self.graph.pheromone[self.current][i] ** self.colony.alpha * \
                    self.eta[self.current][i] ** self.colony.beta * self.graph.matrix[self.current][i] ** 2 / denominator
            
            
            except ValueError:
                pass  # do nothing
            
            
        # select next node by probability roulette
        selected = 0
        
        rand = random.random()
        for i, probability in enumerate(probabilities):
            rand -= probability
            
            if rand <= 0:
                selected = i
                break
        
        
        
        self.allowed.remove(selected)
        self.tabu.append(selected)
        self.total_cost += self.graph.matrix[self.current][selected]
        self.current = selected
        return self.graph.matrix[self.current][selected]

    def _update_pheromone_delta(self):
        
        self.pheromone_delta = [[0 for j in range(self.graph.rank)] for i in range(self.graph.rank)]
        
        for _ in range(1, len(self.tabu)):
            i = self.tabu[_ - 1]
            j = self.tabu[_]
            
            if self.colony.update_strategy == 1:  # ant-quality system
                self.pheromone_delta[i][j] = self.colony.Q
            
            elif self.colony.update_strategy == 2:  # ant-density system - modified
                self.pheromone_delta[i][j] = self.colony.Q * (self.graph.matrix[i][j] ** 2)
            
            else:  # ant-cycle system
                self.pheromone_delta[i][j] = self.colony.Q / self.total_cost
                
                
    def _remember_best_path(self, best_path_till_now):
        
        for _ in range (1, len(best_path_till_now)):
            i = best_path_till_now[_ - 1]
            j = best_path_till_now[_]
            
            self.pheromone_delta[i][j] += self.colony.Q * (self.graph.matrix[i][j] )
        
        
        

### DNA Fragment Assembly Problem using ACO

In [18]:
from plot import plot

def main():
    
    cnt = 0
    row = []
    
    with open('./data/x60189_4.csv') as f:
        
        for line in f.readlines():
            cnt = cnt + 1
            if (cnt > 2):
                temp = line.split(',')
                n = len(temp)
                
                temp [n - 1] = temp [n - 1].replace("\n", "")
                temp = list(map(int, temp))

                row.append(temp)
        
    cost_matrix = row
    
    rank = (len(row))
   
    ant_number = 100
    genarations = 5
    alpha = 10
    beta = 0
    rho = 0.5          
    q = 0.1            
    strategy = 2       # pheromone update strategy 
    
    aco = ACO(ant_number, genarations, alpha, beta, rho, q, strategy)
    
    
    graph = Graph(cost_matrix, rank)
    path, cost = aco.solve(graph)
    
    print('Cost: {}\nPath: {}'.format(cost, path))
    #plot(points, path)



In [19]:
#Maximum
import time

start = time.time()

if __name__ == '__main__':
    main()
    
end = time.time()

print ("Time taken: " + str(end - start))



Cost: 11534.0
Path: [23, 34, 11, 28, 29, 38, 24, 19, 36, 30, 21, 1, 7, 35, 8, 32, 9, 33, 0, 3, 10, 25, 5, 12, 18, 16, 15, 13, 27, 20, 14, 6, 26, 22, 17, 4, 31, 37, 2]
Time taken: 1.570061206817627


### To do - local search after each ant stops, try includeing memory, see lin kernighan algorithm.

### My notes:

To Do: local search after each ant stops, try includeing memory, see lin kernighan algorithm.
Done to improve: Added cost of the nodes in probability of that node being chosen as the next one

Ant Colony Optimization parameters : 




    
        
        Beta          |       Cost(With all else fixed) - average 
    -------------------------------------------------------------------
        10.0          |       21829.620691265285
        5.00          |       23553.221829436254
        3.00          |       25447.203960676263
        2.00          |       31437.522098360576
        1.50          |       36686.737608174630
        1.00          |       41965.959421151616    
        0.75          |       42942.996783108170
        0.50          |       45011.769159315234   -------------------------> lowering more sometimes causes division by zero
        0.25          |       45573.167470364440
        0.10          |       46020.717850682310   
        0.01          |       45925.464856398874
        
    
    
    
    
    
        Alpha         |       Cost(With all else fixed) - average
    ------------------------------------------------------------------
        0.001         |       48048.91786653991
        0.01          |       48112.50137846391       --------------------------> Better
        0.10          |       46355.04555796629
        0.25          |       44597.38538005311
        0.50          |       43499.93387904320
        1.00          |       45011.76915931523
        5.00          |       44444.75636859216
        10.0          |       42185.21637941349
        
        
        
        
      
      
        
        rho         |       Cost(With all else fixed) - average   |     Time
    -----------------------------------------------------------------------------------
        0.75        |       48471.11405497157                     | 4.710163116455078      |        \
        0.50        |       48112.501378463910                    | 4.959774255752563      | ________\   All similar
        0.25        |       47892.152510454005                    | 4.648722410202026      |         /  will continue
        0.10        |       48345.698915634910                    | 4.652569074630737      |        /    using 0.50
        
        



         q          |       Cost(With all else fixed) - average   |     Time
    -----------------------------------------------------------------------------------
         1          |       48481.527575552650                    | 4.734825134277344      |
         5          |       48405.502090066664                    | 4.709308290481567      |        \
        10          |       48112.501378463910                    | 4.959774255752563      | ________\   All similar
        15          |       48489.496508083670                    | 4.671412277221680      |         /  will continue
        20          |       48938.652042056674                    | 4.959913969039917      |        /    using 10
        100         |       48566.103781509850                    | 4.980409860610962      |
        
        
        
        
        
        
        
         stategy    |       Cost(With all else fixed) - average   |     Time
    -----------------------------------------------------------------------------------
         0          |       49107.655625815210                    | 4.395435190200806    ---------------> On average, takes
         1          |       48236.306460781950                    | 4.820995330810547                      lesser time
         2          |       48112.501378463910                    | 4.959774255752563
         
         
         
         
         
         
        
        
      ant_number    |       Cost(With all else fixed) - average  |     Time
    -----------------------------------------------------------------------------------
         1          |       47009.97544730951                    | 0.922459125518798      
         5          |       48394.36481718984                    | 2.784187078475952   -------> Because 1 ant seems too less?  
        10          |       49107.65562581521                    | 4.395435190200806     
        15          |       48429.00596747731                    | 6.036539316177368
        
        
        
        
        
        
        
      genaration    |       Cost(With all else fixed) - average  |     Time
    -----------------------------------------------------------------------------------
         5          |       44693.73623077510                    | 0.452939748764038 
        10          |       46383.64375130145                    | 1.097135066986084   
        50          |       47794.29731875018                    | 1.482169141769409   -------------> Seems better  
        100         |       48394.36481718984                    | 2.784187078475952
        500         |       49227.79728751496                    | 11.19119119644165
          
    
    
    
    '''